https://www.cs.cmu.edu/~fp/courses/15317-s23/lectures/22-invmethod.pdf
https://www.cs.cmu.edu/~fp/courses/99-atp/handouts/fwdseq.pdf
foreward sequent claculus. Gamma |- A   means we needed Gamma


In [ ]:
def subformula(fm):
    seen = set()
    todo = [fm]
    while todo:
        fm = todo.pop()
        if fm in seen:
            continue
        seen.add(fm)
        for subfm in fm.children():
            todo.append(subfm)
    return seen

def inverse(fms):
    seqs = set((frozenset(A), A) for A in fms)
    todo = list(seqs)
    


Sequent calculus is natural deduction where we are buildin trees at top and bottom.

We need zipper continuations to build final tree.


Prolog conitnuation passing style.
We always ave just one thing on the other side.
Maybe it's nice to always have K as first parameter? Or factored out somehow.

Triska had a lovely generic zipper.



In [ ]:
path(X,Z) :- edge(X,Y), path(Y,Z).

edge(1,2,K) :- K.
edge(2,3,K) :- K.

path(X,Y,K) :- edge(X,Y,K).
path(X,Z,K) :- edge(X,Y, path(Y,Z,K)).


% intro
prove(Gam, and(X,Y)) :- !, prove(Gam, X), prove(Gam,Y).
prove(Gam, or(X,Y)) :- prove(Gam,X) ;  prove(Gam,Y).
prove(Gam, X) :- elim(Gam, X).


elim([ | Gam],X) :- member(X, Gam).
elim([or(A,B) | Gam], X) :- prove({A | Gam|, X), prove([B | Gam], X).
elim([and(a,d)  | Gam], X) :- prove([A,B | Gam], X).
elim([A|Gam], X) :- prove([Gam, A], X). % somehow move through the context and do it all


% cps
% Gam = Env ?
prove(Gam, and(X,Y), K) :- prove(Gam,  X,  prove(Gam, Y, K)).


 % tabling via cps?

% chr for forward inference.
:- chr ctx/1

ctx(A), ctx(A) => ctx(A).
ctx(and(A,B))==> ctx(A), ctx(B).
ctx(impl(A,B)) => { ctx(A), prove(B), del(A) }, ctx(B). % but only delete if it isn't in context. This is why linearity might be nice,
ctx(A) | getctx(B) => A = B. 


% multiset rewriting semantics of SNAX


% SAT prolog using two watched literal scheme and delayed wakeup like King et all


Sequent calculus
What is it?


The subformula property


```
goalsubform(A) :- goalsubform(and(A,B))


seq({A}, {A}) :- goalsub(A)

% natrual deduction has isngle right hand consequent
% a natrual guess to
{seq(A,B) :- subgoal(A), subgoal(B)

:- 

:- goal(P), not seq( ,P)

```

Can I drectly use magic set in souffle.

Gam |- 
--------------------

The inverse method is a magic set / demand transformation

Voronkov
Maslov
Pfenning




In [ ]:
"""
need() :- need(Ctx, and(A,B)).
need() :- need(Ctx, or(A,B)).
need() :- need(Ctx, not(A)).


"""

A little bit longer. We got up through the intuitive interpretation of lambda-mu terms that Parigot suggests, how the term for Peirce's law should work, and my attempts to make sense of multiple-conclusion sequents as typing judgements. Didn't get to the proof-normalization procedure he sketches.





Graham Leach-Krouse
  21 hours ago
Yeesh, subformula property was already 2pm? I think I need to work on editing these things down more.


Philip Zucker
  20 hours ago
No you had just barely hit lambda mu


Philip Zucker
  20 hours ago
The subformula property is important to the inverse method, which is a way of pruning what formulas one may need to consider in a bottom up automated method


Philip Zucker
  20 hours ago
I think it's related to the idea of a magic set transformation in datalog


Philip Zucker
  20 hours ago
But I've sort of been too confused to go further (edited) 


Graham Leach-Krouse
  20 hours ago
What's a magic set transformation? (looks like the handbook of automated reasoning has a chapter on the inverse method...)


Philip Zucker
  20 hours ago
frank pfenning also has some notes on inverse method


Philip Zucker
  20 hours ago
I'm not strictly sure what magic set is. But I do get the idea of emulating "demand". I think the motivation is to emulate prolog (a top down / backwards system) with datalog (a bottom up / forward system). You can do this by making "demand" or "magic-set" predicates that track an overapproximation of the possible prolog queries


Philip Zucker
  20 hours ago
Kind of it's like doing some kind of statc analyss / abstract interpretation over prolog programs using datalog


Philip Zucker
  20 hours ago
This comes up for example if you want to compute a function


Philip Zucker
  20 hours ago
fact(0,1). fact(n+1,n*m) :- fact(n, m).


Philip Zucker
  20 hours ago
This is a good prolog program


Philip Zucker
  20 hours ago
But a non terminating datalog one


Philip Zucker
  20 hours ago
demand_fact(n-1) :- demand_fact(n), n > 0.
fact(n+1,n*m) :- fact(n, m), demand_fact(n+1).


Philip Zucker
  20 hours ago
This guards the rule by demand so it stops eventually.

If you want to calculate fact(7,?) you seed the database with demand_fact(7).

Demand rules basically propagates what subfunction calls you need answered in order to answer the current demand.
o
So they tend to push demand to decreasing arguments.

Magic set is a slightly more subtle thing that pushes around which variabes are ground or not.


https://souffle-lang.github.io/magicset

It is a generalization of the above, which is a functional picture with obvious inputs and outputs. Which param in prolog is output vs input depends on which things are grounded
Presumably demand here would be breaking up the formula to be proven into it's subformula
Whose subproofs you desire
And then a straight translation of the rules of inference to datalog rules ought to actualy do the forward part.
I suspect the rub is that there are many possible subformula context to the left of the sequent to ask for
But maybe answer set programming can offer something there
It lets you sprinkle in a little SAT like search into your datalog
Is it obvious when a calculus has the subformula property?
https://www.cs.cmu.edu/~fp/courses/99-atp/lectures/lecture19.html

Can be highly non-obvious. It entails consistency, so there's a sense in which the proofs are intrinsically hard, or at least they require something strong in the background.
https://en.wikipedia.org/wiki/Takeuti%27s_conjecture


but a cut free system its obvious?

Yep, without cuts (in a sequent calculus), it's immediate from inspecting the rules. (edited) 
OK, I'm trying to understand the demand thing, without much logic programming fluency. The idea is that you can by adding demand(7), you can infer demand(n) for n<7, and by mixing demand into the rules for factorial, you prevent divergence caused by endless generation of factorial relation facts?



Put them up here: https://github.com/draperlaboratory/pl-reading-group/tree/master/talks/2024-05-28 - they're still HTML, but they should just work if you open them in a regular browser.

